# Problem Set 3: Minimax & Alpha-beta Pruning

**Release Date:** 8 September 2023

**Due Date:** 23:59, 23 September 2023

## Overview

In class, we discussed a number of search algorithms to implement a two-player game playing agent. In this problem set, we get some hands-on practice by coding an AI to play the game **Breakthrough**.

Breakthrough was the winner of the 2001 8 × 8 Game Design Competition, sponsored by *About.com* and *Abstract Games Magazine*. When Dan Troyka formulated it, it was originally for a 7×7 board. We’re going to play it on a 6×6 board to limit the complexity. In terms of our terminology for the agent environment, Breakthrough is a fully observable, strategic, deterministic game. The game always results in a win for one of the two players.

How exactly do you design an agent to play this game and, most importantly, win? An agent takes sensory input and reasons about it, and then outputs an action at each time step. You thus need to create a program that can read in a representation of the board (that’s the input) and output a legal move in Breakthrough. You then need an evaluation function to evaluate how good a position is to your agent. The better your evaluation function, the better your agent will be at picking good moves.

Aside from the evaluation function, you also need to decide a strategy for exploring the search space. In this problem set, you will first implement a minimax agent, followed by augmenting it with alpha-beta pruning. Additionally, you will be given a limited amount of time to make each move (for the contest) - you must devise a strategy for selecting the optimal move once the allocated search time has expired.

Required Files:

* utils.py
* ps3.py

**Honour Code**: Note that plagiarism will not be condoned! You may discuss with your classmates and check the internet for references, but you MUST NOT submit code/report that is copied directly from other sources!

**IMPORTANT**: While it is possible to write and run Python code directly in Jupyter notebook, we recommend that you do this Problem Set with an IDE using the `.py` file provided. An IDE will make debugging significantly easier.

## Breakthrough Technical Description

<pre>
<p style="text-align: center;">
<img src="imgs/breakthrough_board.png">
Figure 1. Game Board
</p>
</pre>

Figure 1 shows our typical game board. Black (**B**) wins by moving one piece to the opposite side, row index 5. White (**W**) wins by moving one piece to row index 0. A side also wins if their opponent has no pieces left. Kindly **follow the same indexing as provided in *Figure 1*, and write code only for moving black**. A simple board inversion will make black’s code work seamlessly for white as well.

<pre>
<p style="text-align: center;">
<img src="imgs/game_move.png">
Figure 2. Possible Moves
</p>
</pre>

Pieces move one space directly forward or diagonally forward, and only capture diagonally forward. The possible moves have been illustrated in *Figure 2*. In this figure, the black pawn at (3, 2) can go to any of the three spaces indicated forward. The black pawn at (0, 4) can either choose to move by going diagonally right or capture by going diagonally left. It cannot move or capture by moving forward; its forward move is blocked by the white pawn. Note that your move is not allowed to take your pawn outside the board.

<pre>
<p style="text-align: center;">
<img src="imgs/invert_board.png">
Figure 3. Board Inversion Illustration
</p>
</pre>

Your program will always play **black**, whose objective is to move a black pawn to row index 5. Given a move request, your agent should output a pair of coordinates using the coordinate system shown in the figure. For example, for moving the black pawn standing at (0, 4) in *Figure 2* to (1, 3), your agent should make a move that returns two 2 tuples: (0, 4) and (1, 3).

You will implement some basic components to of the agent. Afterward, you can further improve your agent with your own design to compete with agents created by your fellow students in a contest.

## Provided Utility Functions

You can use the functions provided in *util.py* file as you see fit. These functions have mainly been used by the game playing framework to facilitate the two player game. A short description of these functions is given below:

- `generate_init_state()`: It generates the initial state (*Game Board in Figure 1*) at the start of the game.
- `print_state(board)`: It takes in the board 2D list as parameter and prints out the current state of the board in a convenient way (sample shown in *Possible Moves in Figure 2*).
- `is_game_over(board)`: Given a board configuration, it returns `True` if the game is over, `False` otherwise.
- `is_valid_move(board, src, dst)`: It takes in the board configuration and the move source and move destination as its parameters. It returns `True` if the move is valid and returns `False` if the move is invalid.
- `state_change(curr_board, src, dst, in_place=True)`: Given a board configuration and a move source and move destination, this function changes board configuration in accordance to the indicated move. This function updates the board configuration by modifying existing values if `in_place` is set to `True`, or creating a new board with updated values if `in_place` is set to `False`.
- `invert_board(curr_board, in_place=True)`: It takes in the board 2D list as parameter and returns the inverted board. You should always code for black, not for white. The game playing agent has to make move for both black and white using only black’s code. So, when it is time for white to make its move, we invert the board using this function to see everything from white side’s perspective (done by inverting the colors of each pawn and by modifying the row indices). An example of inversion has been shown in *Figure 3 Board Inversion Illustration*. In your minimax algorithm, you need to consider both black and white alternatively. Instead of writing the same code twice separately for black and white, you can use `invert_board()` function to invert your board configuration that enables you to utilize black’s codes for white pawns as well. This function inverts the board by modifying existing values if `in_place` is set to `True`, or creating a new board with updated values if `in_place` is set to `False`.
- `generate_rand_move(board)`: It takes in the board configuration as its parameter and generates an arbitrary valid move. You likely won’t need to use this function. This function is used by the game playing framework in one of two cases - (1) an invalid move has been made by the game playing agent or (2) the game playing agent has taken more than 3 seconds to make its move.

Other functions are used to play the game or test your solution - you don't need to use those functions.

In [ ]:
'''
Run this cell before you start!
'''
import utils

Score = int | float
Move = tuple[tuple[int, int], tuple[int, int]]

To build your own agent, you will need a heuristic function to evaluate a position. One sample heuristic function is provided below.

In [ ]:
# remember, we are black
def evaluate(board):
    bcount = 0
    wcount = 0
    for r, row in enumerate(board):
        for tile in row:
            if tile == 'B':
                if r == 5:
                    return utils.WIN
                bcount += 1
            elif tile == 'W':
                if r == 0:
                    return -utils.WIN
                wcount += 1
    if wcount == 0:
        return utils.WIN
    if bcount == 0:
        return -utils.WIN
    return bcount - wcount

The provided heuristic function returns `utils.WIN` if black wins, and `-utils.WIN` if white wins. Otherwise, it takes the difference between the number of black pieces and the number of white pieces that are on the board. 

**Note**: On Coursemology, we will provide and use this heuristic function to test your code in task 2 and task 3.

### Task 1.1: Implement a function to generate all valid moves

It is useful to generate all the possible moves that black can make in a certain position. You may need this function when implementing the minimax algorithm.

**Note**: On Coursemology, we will provide you with the correct implementation of `generate_valid_moves` in task 2 and task 3.

In [ ]:
def generate_valid_moves(board):
    '''
    Generates a list (or iterable, if you want to) containing 
    all possible moves in a particular position for black.
    '''
    # TODO: Replace this with your own implementation
    raise NotImplementedError

In [ ]:
# Note that order of the moves might be different

board1 = [
    ['_', '_', 'B', '_', '_', '_'],
    ['_', '_', '_', '_', '_', '_'],
    ['_', '_', '_', '_', '_', '_'],
    ['_', '_', '_', '_', '_', '_'],
    ['_', '_', '_', '_', '_', '_'],
    ['_', '_', '_', '_', '_', '_']
]
assert sorted(generate_valid_moves(board1)) == [((0, 2), (1, 1)), ((0, 2), (1, 2)), ((0, 2), (1, 3))]

board2 = [
    ['_', '_', '_', '_', 'B', '_'],
    ['_', '_', '_', '_', '_', 'B'],
    ['_', 'W', '_', '_', '_', '_'],
    ['_', '_', 'W', '_', '_', '_'],
    ['_', '_', '_', '_', '_', '_'],
    ['_', '_', 'B', '_', '_', '_']
]
assert sorted(generate_valid_moves(board2)) == [((0, 4), (1, 3)), ((0, 4), (1, 4)), ((1, 5), (2, 4)), ((1, 5), (2, 5))]

board3 = [
    ['_', '_', '_', '_', '_', '_'],
    ['_', '_', 'B', '_', '_', '_'],
    ['_', 'W', 'W', 'W', '_', '_'],
    ['_', '_', '_', '_', '_', '_'],
    ['_', '_', '_', '_', '_', '_'],
    ['_', '_', '_', '_', '_', '_']
]
assert sorted(generate_valid_moves(board3)) == [((1, 2), (2, 1)), ((1, 2), (2, 3))]

## Minimax Algorithm

Your agent must be able to calculate the game state a few moves in advance, by implementing the **minimax** algorithm.

### Task 2.1: Implement minimax

In the lecture, you have seen the minimax algorithm without and with cutoff. We will implement a minimax algorithm with cutoff in this case, as the depth of the game and the branching factor in certain positions can be (very) large and we do not have the computational power to compute the entire game until the terminal states.

Your minimax function should explore different game states, until either the depth is `max_depth`, or there is a winner. In these cases, your minimax algorithm should use the provided heuristic function to evaluate the position.

You can reuse `generate_valid_moves` and `evaluate` to handle the white side if you use `invert_board` from `utils.py`. If you choose to do this, remember to invert the board again when you need to handle the black side.

**Note**: For tasks 2.1 to 3.2, if you are certain that your solution is correct but the test cases fail on Coursemology due to timeout, just rerun your code. Depending on the load on Coursemology, a correct solution might still timeout.

In [ ]:
def minimax(board, depth, max_depth, is_black: bool) -> tuple[Score, Move]:
    '''
    Finds the best move for the input board state.
    Note that you are black.

    Parameters
    ----------
    board: 2D list of lists. Contains characters 'B', 'W' and '_' 
    representing black pawn, white pawn and empty cell, respectively.
    
    depth: int, the depth to search for the best move. When this is equal
    to `max_depth`, you should get the evaluation of the position using 
    the provided heuristic function.

    max_depth: int, the maximum depth for cutoff.
    
    is_black: bool. True when finding the best move for black, False 
    otherwise.

    Returns
    -------
    A tuple (evalutation, ((src_row, src_col), (dst_row, dst_col))):
    evaluation: the best score that black can achieve after this move.
    src_row, src_col: position of the pawn to move.
    dst_row, dst_col: position to move the pawn to.
    '''
    # TODO: relace with your own implementation
    raise NotImplementedError

In [ ]:
# Note that there can be multiple best moves, denoted by _

board1 = [
    list("______"),
    list("___B__"),
    list("____BB"),
    list("___WB_"),
    list("_B__WW"),
    list("_WW___"),
]
score1, _ = minimax(board1, 0, 1, True)
assert score1 == utils.WIN, "black should win in 1"

board2 = [
    list("______"),
    list("___B__"),
    list("____BB"),
    list("_BW_B_"),
    list("____WW"),
    list("_WW___"),
]
score2, _ = minimax(board2, 0, 3, True)
assert score2 == utils.WIN, "black should win in 3"

board3 = [
    list("______"),
    list("__B___"),
    list("_WWW__"),
    list("______"),
    list("______"),
    list("______"),
]
score3, _ = minimax(board3, 0, 4, True)
assert score3 == -utils.WIN, "white should win in 4"

### Task 2.2: Implement negamax

You may notice that Breakthrough is a zero-sum game. It means that the sum of the evalutation scores of the two players should be zero.

For example, we can consider a position in which there are _9 black pawns_ and _6 white pawns_. Using the sample heuristic function given at the start:

- The evaluation score of black in this position is `+3`.
- The evaluation score of white in this position is `-3`.

Using this property, we can simplify the implementation of minimax. Instead of taking the maximum and minimum scores for black and white respectively, we can negate the score of the opposite player and take the maximum score. This version is called **negamax**.

In [ ]:
def negamax(board, depth, max_depth) -> tuple[Score, Move]:
    '''
    Finds the best move for the input board state.
    Note that you are black.

    Parameters
    ----------
    board: 2D list of lists. Contains characters 'B', 'W' and '_' 
    representing black pawn, white pawn and empty cell, respectively.
    
    depth: int, the depth to search for the best move. When this is equal
    to `max_depth`, you should get the evaluation of the position using 
    the provided heuristic function.

    max_depth: int, the maximum depth for cutoff.

    Notice that you no longer need the parameter `is_black`.

    Returns
    -------
    A tuple (evalutation, ((src_row, src_col), (dst_row, dst_col))):
    evaluation: the best score that black can achieve after this move.
    src_row, src_col: position of the pawn to move.
    dst_row, dst_col: position to move the pawn to.
    '''
    # TODO: replace with your own implementation
    raise NotImplementedError

In [ ]:
# Note that there can be multiple best moves, denoted by _

board1 = [
    list("______"),
    list("___B__"),
    list("____BB"),
    list("___WB_"),
    list("_B__WW"),
    list("_WW___"),
]
score1, _ = negamax(board1, 0, 1)
assert score1 == utils.WIN, "black should win in 1"

board2 = [
    list("______"),
    list("___B__"),
    list("____BB"),
    list("_BW_B_"),
    list("____WW"),
    list("_WW___"),
]
score2, _ = negamax(board2, 0, 3)
assert score2 == utils.WIN, "black should win in 3"

board3 = [
    list("______"),
    list("__B___"),
    list("_WWW__"),
    list("______"),
    list("______"),
    list("______"),
]
score3, _ = negamax(board3, 0, 4)
assert score3 == -utils.WIN, "white should win in 4"

If you implement negamax correctly, the code should be much more elegant compared to minimax.

## Alpha-beta Pruning

With minimax (or negamax), our agent can see the future within a few moves. However, the naive implementation of minimax (or negamax) may explore many redundant states, which slows down our agent. As discussed in the lecture, we can apply **alpha-beta pruning** to eliminate unnecessary states, thereby improving our agent's speed and its ability to see even further into the future. This will increase our agent's strength and its likelihood of winning the game. 

First, you should try to integrate alpha-beta pruning with the standard minimax algorithm.

### Task 3.1: Integrate alpha-beta pruning into minimax

In [ ]:
def minimax_alpha_beta(board, depth, max_depth, alpha, beta, is_black: bool) -> tuple[Score, Move]:
    # TODO: Replace this with your own implementation
    raise NotImplementedError

In [ ]:
# Note that there can be multiple best moves, denoted by _

board1 = [
    list("______"),
    list("__BB__"),
    list("____BB"),
    list("WBW_B_"),
    list("____WW"),
    list("_WW___"),
]
score1, _ = minimax_alpha_beta(board1, 0, 3, -utils.INF, utils.INF, True)
assert score1 == utils.WIN, "black should win in 3"

board2 = [
    list("____B_"),
    list("___B__"),
    list("__B___"),
    list("_WWW__"),
    list("____W_"),
    list("______"),
]
score2, _ = minimax_alpha_beta(board2, 0, 5, -utils.INF, utils.INF, True)
assert score2 == utils.WIN, "black should win in 5"

board3 = [
    list("____B_"),
    list("__BB__"),
    list("______"),
    list("_WWW__"),
    list("____W_"),
    list("______"),
]
score3, _ = minimax_alpha_beta(board3, 0, 6, -utils.INF, utils.INF, True)
assert score3 == -utils.WIN, "white should win in 6"

### Task 3.2: Integrate alpha-beta pruning into negamax

At this stage, you may wonder: why don't we integrate alpha-beta pruning with the more elegant alternative, negamax? Of course, we can also incorporate alpha-beta pruning into our negamax algorithm to improve its performance.

Remember, we exploited the zero-sum property of the game to implement negamax by negating the evaluation score, and always taking the maximum score instead of alternating between the maximum and minimum. You need to further exploit this property to correctly integrate alpha-beta pruning into negamax. To help you, these are some questions that you can try to answer:

- What is the meaning of `alpha` and `beta`?
- From the perspective of the opponent, what is the corresponding `alpha` and `beta`? Can I exploit the zero-sum property here?

In [ ]:
def negamax_alpha_beta(board, depth, max_depth, alpha, beta) -> tuple[Score, Move]:
    # TODO: Replace this with your own implementation
    raise NotImplementedError

In [ ]:
# Note that there can be multiple best moves, denoted by _

board1 = [
    list("______"),
    list("__BB__"),
    list("____BB"),
    list("WBW_B_"),
    list("____WW"),
    list("_WW___"),
]
score1, _ = negamax_alpha_beta(board1, 0, 3, -utils.INF, utils.INF)
assert score1 == utils.WIN, "black should win in 3"

board2 = [
    list("____B_"),
    list("___B__"),
    list("__B___"),
    list("_WWW__"),
    list("____W_"),
    list("______"),
]
score2, _ = negamax_alpha_beta(board2, 0, 5, -utils.INF, utils.INF)
assert score2 == utils.WIN, "black should win in 5"

board3 = [
    list("____B_"),
    list("__BB__"),
    list("______"),
    list("_WWW__"),
    list("____W_"),
    list("______"),
]
score3, _ = negamax_alpha_beta(board3, 0, 6, -utils.INF, utils.INF)
assert score3 == -utils.WIN, "white should win in 6"

## Heuristic Function

Phew, we finish the search algorithm! But, our heuristic function is too simple - it may not give the best evaluation for a position and we need a better one. Therefore, you shall implement the not-as-simple heuristic function described below.

### Task 4.1: Implement a less simple heuristic function

Recall that the heuristic function should return a larger value when black is closer to winning. If black is closer to winning, black should have more pieces closer to row 5 compared to white having pieces closer to row 0. Of course this is not necessarily the case since you only need one piece to make it through while the rest remain behind, but this is just a heuristic after all. Thus, in this heuristic you are about to implement, we add more points if a black piece is closer to the end, and subtract more points if a white piece is closer to the end. The exact amount of points is shown in the figures below.

<pre>
<p style="text-align: center;">
<img src="imgs/black_heuristic.png", width = 300>
Figure 4. Points to add for each black piece in the square
<p style="text-align: center;">
<img src="imgs/white_heuristic.png", width = 300>
Figure 5. Points to subtract for each white piece in the square
</p>
</pre>

So, for example, if there are two black pieces on (0, 4) and (3, 2), and a white piece on (1, 4), then the output of the heuristic function should be `10 + 40 - 50 = 0`. Additionally, return `utils.WIN` if any of black's pieces reach the end, and `-utils.WIN` if any of white's pieces reach the end. Similarly, if white has no pieces, return `utils.WIN`, and if black has no pieces, return `-utils.WIN`. The value of `utils.WIN` can be found in `utils.py` and has a value of `101010`.

In [ ]:
def evaluate(board):
    # TODO: replace this with your own implementation
    raise NotImplementedError

In [ ]:
board1 = [
    ['_', '_', '_', 'B', '_', '_'],
    ['_', '_', '_', 'W', '_', '_'],
    ['_', '_', '_', '_', '_', '_'],
    ['_', '_', 'B', '_', '_', '_'],
    ['_', '_', '_', '_', '_', '_'],
    ['_', '_', '_', '_', '_', '_']
]
assert evaluate(board1) == 0

board2 = [
    ['_', '_', '_', 'B', 'W', '_'],
    ['_', '_', '_', 'W', '_', '_'],
    ['_', '_', '_', '_', '_', '_'],
    ['_', '_', '_', '_', '_', '_'],
    ['_', '_', '_', '_', '_', '_'],
    ['_', '_', '_', '_', '_', '_']
]
assert evaluate(board2) == -utils.WIN

board3 = [
    ['_', '_', '_', '_', '_', '_'],
    ['_', '_', '_', '_', '_', '_'],
    ['_', '_', '_', '_', '_', '_'],
    ['_', '_', 'B', '_', '_', '_'],
    ['_', '_', '_', '_', '_', '_'],
    ['_', '_', '_', '_', '_', '_']
]
assert evaluate(board3) == utils.WIN

## Free Implementation and Contest

Finally, you can combine the implemented components together and create your own agent. But wait, there is something missing - we must deal with the time constraint! Your agent only has a limited amount of time for calculation before making a move.

Your agent _**must not take more than 3 real-time seconds**_ to make a move in the contest. You should check for time passed during every recursive call in your algorithm to follow this 3 second rule. Whenever you see that 3 seconds is almost over, immediately return the best move you have at your disposal. This is really important because the machine where we will run your code may be much slower than your local machine.

In [ ]:
class PlayerAI:

    def make_move(self, board) -> Move:
        '''
        This is the function that will be called from main.py
        You should combine the functions in the previous tasks
        to implement this function.

        Parameters
        ----------
        self: object instance itself, passed in automatically by Python.
        
        board: 2D list-of-lists. Contains characters 'B', 'W', and '_' 
        representing black pawn, white pawn and empty cell respectively.
        
        Returns
        -------
        Two tuples of coordinates [row_index, col_index].
        The first tuple contains the source position of the black pawn
        to be moved, the second list contains the destination position.
        '''
        # TODO: Replace starter code with your AI
        ################
        # Starter code #
        ################
        for r in range(len(board)):
            for c in range(len(board[r])):
                # check if B can move forward directly
                if board[r][c] == 'B' and board[r + 1][c] == '_':
                    src = r, c
                    dst = r + 1, c
                    return src, dst # valid move
        return (0, 0), (0, 0) # invalid move

After this, you are free to further improve your agent with any technique, except for some that allow you to gain unfair advantages, including, but not limited to:
- Change the testing framework / timer.
- Use Python to compile C++ as this is hardware advantage.
- Use of multi-process as this is hardware advantage.

The maximum size of code (of your agent) that you can upload is 10MB.

Ultimately, we shall be playing all the student designed agents against each other. So, it will be a small Breakthrough tournament. The top players will get some bonus XP.

## Testing Your Game Playing Agent

Fill in `make_move(board)` method of the `PlayerAI` class with your game playing agent code. The `PlayerNaive` class has been provided for you to test out your agent against another program. Always code for black (assume black as max player) in both these class functions. The game playing framework calls the `make_move(board)` method of each agent alternatively. After you complete `PlayerAI`, simply run the *template.py* file. You will see the two agents (`PlayerAI` and `PlayerNaive`) playing against each other.

Your agent should always provide a legal move. Moves will be validated by the game playing framework. If your player makes an illegal move, the competition framework will choose the next available valid move on your behalf, so you will likely lose. Your agent must always make a move; it is not allowed to skip moves. Your program *cannot take more than 3 real-time seconds* to make a move. If your program does not output a coordinate within 3 seconds, the competition framework will choose the next available move too. You can read up the implementation to obtain the next available move by looking up the function `generate_rand_move` in `utils.py`.

To maximise your chances of winning, you might want to optimise the following points:
- The evaluation function used to evaluate a certain position.
- Effective exploration strategy (for example: move ordering).
- Modifying the alpha-beta pruning algorithm for more efficient search.

# Submission

Once you are done, please submit your work to Coursemology, by copying the right snippets of code into the corresponding box that says 'Your answer', and click 'Save'.  After you save, you can make changes to your
submission.

Once you are satisfied with what you have uploaded, click 'Finalize submission.'  **Note that once your submission is finalized, it is considered to be submitted for grading and cannot be changed**. If you need to undo
this action, you will have to email your assigned tutor for help. Please do not finalize your submission until you are sure that you want to submit your solutions for grading. 
